In [ ]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-i6qnz5mu
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-i6qnz5mu
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-d6h979vk/pyfolio_b6efe478f343483f81e15adf455ed380
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-d6h979vk/pyfolio_b6efe478f343483f81e15adf455ed380
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-d6h979vk/elegantrl_09ede66268264aaa952f04e77ab23642
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-d6h979vk/elegantrl_09ede66268264aaa952f04e77ab23642
     |████████████████████████████████| 234 kB 45.9 MB/s 
     |████████████████████████████████| 177 kB 18.8 MB/s 
     |█████████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/STECH/Reinforcement Learning1

/content/drive/MyDrive/STECH/Reinforcement Learning1


In [ ]:
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists('./' + config.DATA_SAVE_DIR):
  os.makedirs('./' + config.DATA_SAVE_DIR)
if not os.path.exists('./' + config.TRAINED_MODEL_DIR):
  os.makedirs('./' + config.TRAINED_MODEL_DIR)
if not os.path.exists('./' + config.TENSORBOARD_LOG_DIR):
  os.makedirs('./' + config.TENSORBOARD_LOG_DIR)
if not os.path.exists('./' + config.RESULTS_DIR):
  os.makedirs('./' + config.RESULTS_DIR)

In [ ]:
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

# import sys
# sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%cd datasets
#FX_IDC_USDGBP
!gdown --id 1rgtkD35D_XxwFoovGLFom3_ojRvPFyeI
#FX_IDC_USDAUD 1DAY
!gdown --id 149sQHIF-il4iip76x7UhrLP88o16JYRg
#FX_USDCAD  1DAY
!gdown --id 1FFTzUMR9-1umCQlH3bp453SHXZFSWzCp
#FX_USDCHF
!gdown --id 1nD4iVk-A0PAFEjJ_ziMHRRTngHKMZt-K
#FX_EURUSD, 1D
!gdown --id 15GiISZwR8EM9SsLVd_gobQIDTikpZZcQ  
#FX_USDJPY, 1D
!gdown --id 1_F0go2NXmTfMx-YXuQFEY2kGq0sCAlHG
#FX_EURAUD, 1D
!gdown --id 1dFAbrS9KQLzse591GvFQZsYSSvQqgmNy

/content/drive/MyDrive/STECH/Reinforcement Learning1/datasets
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1rgtkD35D_XxwFoovGLFom3_ojRvPFyeI
To: /content/drive/MyDrive/STECH/Reinforcement Learning1/datasets/FX_IDC_USDGBP, 1D (1).csv
100% 4.10M/4.10M [00:00<00:00, 209MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=149sQHIF-il4iip76x7UhrLP88o16JYRg
To: /content/drive/MyDrive/STECH/Reinforcement Learning1/datasets/FX_IDC_USDAUD, 1D.csv
100% 4.02M/4.02M [00:00<00:00, 223MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: F

In [ ]:
df1 = pd.read_csv('FX_EURAUD, 1D.csv')
df2 = pd.read_csv('FX_IDC_USDAUD, 1D.csv')
df3 = pd.read_csv('FX_IDC_USDGBP, 1D (1).csv')
df4 = pd.read_csv('FX_USDCAD, 1D.csv')
df5 = pd.read_csv('FX_USDCHF, 1D.csv')
df6 = pd.read_csv('FX_USDJPY, 1D.csv')

In [ ]:
df1.head()

,time,open,high,low,close,MA,Smoothing Line,EMA,Smoothing Line.1,DEMA,Volume,Volume MA
0,1993-05-09T21:00:00Z,1.75272,1.77199,1.75272,1.76307,NaN,NaN,NaN,NaN,NaN,0,NaN
1,1993-05-10T21:00:00Z,1.76307,1.77368,1.75904,1.75904,NaN,NaN,NaN,NaN,NaN,0,NaN
2,1993-05-11T21:00:00Z,1.75904,1.76988,1.74380,1.74380,NaN,NaN,NaN,NaN,NaN,0,NaN
3,1993-05-12T21:00:00Z,1.74380,1.75541,1.73909,1.75541,NaN,NaN,NaN,NaN,NaN,0,NaN
4,1993-05-13T21:00:00Z,1.75541,1.76368,1.74637,1.74637,NaN,NaN,NaN,NaN,NaN,0,NaN


In [ ]:
def transformdata2 (df, ticname):
  df['time'] = pd.to_datetime(df['time'])
  df = df.rename(columns={'time':'date','Volume':'volume'})
  df['day'] = df['date'].dt.dayofweek
  df['date'] = df['date'].dt.date
  df['tic'] = ticname
  df = df[['date', 'open', 'high','low','close', 'volume','tic','day']]
  return df

In [ ]:
df1trans = transformdata2 (df1, 'EURAUD')
df2trans = transformdata2 (df2, 'USDAUD')
df3trans = transformdata2 (df3, 'USDGBP')
df4trans = transformdata2 (df4, 'USDCAD')
df5trans = transformdata2 (df5, 'USDCHF')
df6trans = transformdata2 (df6, 'USDJPY')

In [ ]:
df = pd.concat([df1trans
                , df2trans
                , df3trans, df4trans, df5trans, df6trans
                ])

In [ ]:
df.head()

,date,open,high,low,close,volume,tic,day
0,1993-05-09,1.75272,1.77199,1.75272,1.76307,0,EURAUD,6
1,1993-05-10,1.76307,1.77368,1.75904,1.75904,0,EURAUD,0
2,1993-05-11,1.75904,1.76988,1.74380,1.74380,0,EURAUD,1
3,1993-05-12,1.74380,1.75541,1.73909,1.75541,0,EURAUD,2
4,1993-05-13,1.75541,1.76368,1.74637,1.74637,0,EURAUD,3


In [ ]:
def preprocessing_data(df, n_tics = 5):
  df = df[df.groupby('date').date.transform('count')>n_tics].copy() 
  df = df.reset_index()
  df = df.replace([np.inf, -np.inf], 0)
  return df

In [ ]:
df = preprocessing_data(df)
print(df)

       index        date       open       high        low      close   volume  \
0          1  1993-05-10    1.76307    1.77368    1.75904    1.75904        0   
1          2  1993-05-11    1.75904    1.76988    1.74380    1.74380        0   
2          3  1993-05-12    1.74380    1.75541    1.73909    1.75541        0   
3          4  1993-05-13    1.75541    1.76368    1.74637    1.74637        0   
4          6  1993-05-17    1.74041    1.77380    1.74041    1.76738        0   
...      ...         ...        ...        ...        ...        ...      ...   
35713   7534  2022-05-10  130.42900  130.81200  129.44500  129.93900   508428   
35714   7535  2022-05-11  129.93900  130.05400  127.51600  128.26300  1259196   
35715   7536  2022-05-12  128.26300  129.45400  128.24800  129.24500   450358   
35716   7538  2022-05-16  129.11300  129.77900  128.82700  129.35600   335378   
35717   7539  2022-05-17  129.35600  129.53600  128.94100  129.13000   152327   

          tic  day  
0     

In [ ]:
df = df[['date', 'open', 'high','low','close', 'volume','tic','day']]

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35718 entries, 0 to 35717
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    35718 non-null  datetime64[ns]
 1   open    35718 non-null  float64       
 2   high    35718 non-null  float64       
 3   low     35718 non-null  float64       
 4   close   35718 non-null  float64       
 5   volume  35718 non-null  int64         
 6   tic     35718 non-null  object        
 7   day     35718 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 2.5+ MB


In [ ]:
df.to_csv('transform.csv',index=False)
df = pd.read_csv('transform.csv')

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (7309, 8)
Successfully added vix
Successfully added turbulence index


In [ ]:
list_ticker = processed['tic'].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))

In [ ]:
combination

In [ ]:
processed_full = pd.DataFrame(combination, columns=['date','tic']).merge(processed, on=['date','tic'],how='left')

In [ ]:
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,1993-05-10,EURAUD,1.76307,1.77368,1.75904,1.75904,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.759040,1.759040,13.720000,0.000000
1,1993-05-10,USDAUD,1.41110,1.42720,1.41040,1.42370,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.423700,1.423700,13.720000,0.000000
2,1993-05-10,USDCAD,1.26830,1.27110,1.26610,1.27110,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.271100,1.271100,13.720000,0.000000
3,1993-05-10,USDCHF,1.42710,1.46090,1.42650,1.45860,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.458600,1.458600,13.720000,0.000000
4,1993-05-10,USDGBP,0.63840,0.65320,0.63800,0.65130,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,0.651300,0.651300,13.720000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63589,2022-05-16,USDAUD,1.44100,1.45476,1.43234,1.43390,0.0,0.0,0.022582,1.470520,1.316685,58.366845,120.064722,19.295117,1.373169,1.376788,27.469999,13.566394
63590,2022-05-16,USDCAD,1.29163,1.29814,1.28362,1.28449,279373.0,0.0,0.008995,1.310764,1.243793,53.756782,82.904547,0.330439,1.268975,1.269934,27.469999,13.566394
63591,2022-05-16,USDCHF,1.00131,1.00643,1.00072,1.00176,148198.0,0.0,0.017009,1.013379,0.923431,73.113053,142.765555,73.491745,0.955353,0.940382,27.469999,13.566394
63592,2022-05-16,USDGBP,0.81549,0.81848,0.81113,0.81160,0.0,0.0,0.013895,0.831083,0.750746,66.969186,115.218993,47.222252,0.781715,0.764825,27.469999,13.566394


In [ ]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]

In [ ]:
processed_full = processed_full.sort_values(['date','tic'])

In [ ]:
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,1993-05-10,EURAUD,1.76307,1.77368,1.75904,1.75904,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.759040,1.759040,13.720000,0.000000
1,1993-05-10,USDAUD,1.41110,1.42720,1.41040,1.42370,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.423700,1.423700,13.720000,0.000000
2,1993-05-10,USDCAD,1.26830,1.27110,1.26610,1.27110,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.271100,1.271100,13.720000,0.000000
3,1993-05-10,USDCHF,1.42710,1.46090,1.42650,1.45860,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.458600,1.458600,13.720000,0.000000
4,1993-05-10,USDGBP,0.63840,0.65320,0.63800,0.65130,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,0.651300,0.651300,13.720000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63589,2022-05-16,USDAUD,1.44100,1.45476,1.43234,1.43390,0.0,0.0,0.022582,1.470520,1.316685,58.366845,120.064722,19.295117,1.373169,1.376788,27.469999,13.566394
63590,2022-05-16,USDCAD,1.29163,1.29814,1.28362,1.28449,279373.0,0.0,0.008995,1.310764,1.243793,53.756782,82.904547,0.330439,1.268975,1.269934,27.469999,13.566394
63591,2022-05-16,USDCHF,1.00131,1.00643,1.00072,1.00176,148198.0,0.0,0.017009,1.013379,0.923431,73.113053,142.765555,73.491745,0.955353,0.940382,27.469999,13.566394
63592,2022-05-16,USDGBP,0.81549,0.81848,0.81113,0.81160,0.0,0.0,0.013895,0.831083,0.750746,66.969186,115.218993,47.222252,0.781715,0.764825,27.469999,13.566394


In [ ]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))

13782
1614


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension

In [ ]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

In [ ]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [ ]:
e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cuda device
